In [3]:
a = [1,2]
b = ['x','y']
for A,B in zip(a,b):
    print(A)
    print(B)

1
x
2
y


In [4]:
NAME = ['hinge','logistic']
ALG = ['SAUC','OAM']
for name in NAME:
    for alg in ALG:
        a[(name,alg)] = len(name)

In [5]:
a

{('hinge', 'OAM'): 5,
 ('hinge', 'SAUC'): 5,
 ('logistic', 'OAM'): 8,
 ('logistic', 'SAUC'): 8,
 'hinge': 5,
 'logistic': 8}

In [3]:
import numpy as np
import time

In [38]:
n = np.arange(10000)+2
log2 = lambda x: log(x)/log(2)
m = np.zeros(10000)
for i in range(10000):
    m[i] = floor(log2(2*n[i]/log2(n[i]))/2) - 1

In [5]:
y = 1
a = np.random.rand(100)
b = np.random.rand(100)
c = np.random.rand(100)

In [6]:
start = time.time()
if y == 1:
    x = np.inner(a,b) 
else:
    x = np.inner(b,c)
print(time.time() - start)

0.00026869773864746094


In [7]:
start = time.time()
x = np.inner(a,b) * (1+y)//2 + np.inner(b,c) * (1-y)//2
print(time.time() - start)

0.0003979206085205078


In [ ]:
comb_dict = {0:{0:1},1:{0:1,1:1},2:{0:1,1:2,2:1},3:{0:1,1:3,2:3,3:1},4:{0:1,1:4,2:6,3:4,4:1},
             5:{0:1,1:5,2:10,3:10,4:5,5:1},6:{0:1,1:6,2:15,3:20,4:15,5:6,6:1},
             7:{0:1,1:7,2:21,3:35,4:35,5:21,6:7,7:1},8:{0:1,1:8,2:28,3:56,4:70,5:56,6:28,7:8,8:1},
             9:{0:1,1:9,2:36,3:84,4:126,5:126,6:84,7:36,8:9,9:1},
             10:{0:1,1:10,2:45,3:120,4:210,5:252,6:210,7:120,8:45,9:10,10:1}}